In [1]:
import os
import openai
from langchain.document_loaders.unstructured import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
import numpy as np

In [2]:
openai.api_key = os.getenv("OPENAI_API_KEY")

## Load Document:

In [ ]:
loader = UnstructuredFileLoader('documents/codicedellastrada.pdf')
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [3]:
#sanity check
texts[150].page_content

'7.\n\nLa somma dovuta per l’uso o l’occupazione delle strade e delle loro pertinenze può essere stabilita dall’ente proprietario della strada in annualità ovvero in unica soluzione.\n\n8. Nel determinare la misura della somma si ha riguardo alle soggezioni che derivano alla strada o autostrada, quando la concessione costituisce l’oggetto principale dell’impresa, al valore economico risultante dal provvedimento di autorizzazione o concessione e al vantaggio che l’utente ne ricava.\n\n9.\n\nL’autorità competente al rilascio dei provvedimenti autorizzatori di cui al presente titolo può chiedere un deposito cauzionale.'

In [9]:
len(texts[162].page_content)

991

## Embed Text Chunks:

In [11]:
embedding_function = OpenAIEmbeddings()

In [6]:
db = Chroma.from_documents(texts, embedding_function, persist_directory="knowledge/")
db.persist()

100%|██████████| 1/1 [00:05<00:00,  5.17s/it]


In [12]:
#load db
db=Chroma(persist_directory="knowledge/", embedding_function=embedding_function)

## Run Queries:

In [6]:
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=db.as_retriever())

In [18]:
qa.run('Cosa si intende per strada a senso unico?')

' Una strada a senso unico è una strada su cui i veicoli possono viaggiare in una sola direzione.'

In [20]:
qa.run('Quali sono le sanzioni in caso di eccesso di velocià?')

" In caso di eccesso di velocità, la sanzione amministrativa pecuniaria varia da €39,00 a €3.119,00 a seconda dell'entità dell'eccesso. La sanzione amministrativa accessoria prevede la sospensione della patente di guida da uno a dodici mesi."

In [21]:
qa.run('What is the fine in case of speeding?')

' It depends on how much the speed limit is exceeded. If the speed limit is exceeded by up to 10 km/h, the fine is between €39 and €159. If the speed limit is exceeded by between 10 km/h and 40 km/h, the fine is between €159 and €639. If the speed limit is exceeded by between 40 km/h and 60 km/h, the fine is between €500 and €2,000. If the speed limit is exceeded by more than 60 km/h, the fine is between €779 and €3,119.'

In [9]:
qa.run('In which situations it is allowed to not wear a seatbelt?')

' According to Article 172 of the Italian driving code, it is allowed to not wear a seatbelt in the following situations: for members of the police force and municipal police carrying out an emergency service, for drivers and attendants of fire and ambulance services in case of emergency, for children under three years old travelling in vehicles with front airbags unless the airbag has been deactivated, and for children under 1.5 meters tall travelling in public service vehicles or vehicles for hire with a driver, provided they are accompanied by a passenger aged 16 or over.'

In [18]:
qa.run('What is the speed limit in urban areas?')

' 50 km/h'

## Embedding Analysis:

In [13]:
#explore the content of the db
db.get()

{'ids': ['7a36234a-1cb2-11ee-9457-fb11816db9a1',
  '7a36233c-1cb2-11ee-9457-fb11816db9a1',
  '7a36233d-1cb2-11ee-9457-fb11816db9a1',
  '7a36233e-1cb2-11ee-9457-fb11816db9a1',
  '7a36233f-1cb2-11ee-9457-fb11816db9a1',
  '7a362340-1cb2-11ee-9457-fb11816db9a1',
  '7a362341-1cb2-11ee-9457-fb11816db9a1',
  '7a362342-1cb2-11ee-9457-fb11816db9a1',
  '7a362343-1cb2-11ee-9457-fb11816db9a1',
  '7a362344-1cb2-11ee-9457-fb11816db9a1',
  '7a362345-1cb2-11ee-9457-fb11816db9a1',
  '7a362346-1cb2-11ee-9457-fb11816db9a1',
  '7a362347-1cb2-11ee-9457-fb11816db9a1',
  '7a362348-1cb2-11ee-9457-fb11816db9a1',
  '7a362349-1cb2-11ee-9457-fb11816db9a1',
  '7a36233b-1cb2-11ee-9457-fb11816db9a1',
  '7a36234b-1cb2-11ee-9457-fb11816db9a1',
  '7a36234c-1cb2-11ee-9457-fb11816db9a1',
  '7a36234d-1cb2-11ee-9457-fb11816db9a1',
  '7a36234e-1cb2-11ee-9457-fb11816db9a1',
  '7a36234f-1cb2-11ee-9457-fb11816db9a1',
  '7a362350-1cb2-11ee-9457-fb11816db9a1',
  '7a362351-1cb2-11ee-9457-fb11816db9a1',
  '7a362352-1cb2-11ee-9457-

In [14]:
#in order to see the embeddings, add "include=['embeddings']"
embeddings=np.asarray(db.get(include=['embeddings'])['embeddings'])
embeddings.shape

(938, 1536)

In [26]:
print(embeddings[10:12])

[[-0.01491894  0.00957309  0.00425001 ...  0.01969248 -0.02558462
  -0.00311191]
 [-0.01791039  0.00221909  0.00470427 ...  0.01587363 -0.01587363
   0.00166801]]


## QA Object Analysis:

In [13]:
db.similarity_search('What is the speed limit on motorways?')

[Document(page_content='1. Ai fini della sicurezza della circolazione e della tutela della vita umana la velocità massima non può superare i 130 km/h per le autostrade, i 110 km/h per le strade extraurbane principali, i 90 km/h per le strade extraurbane secondarie e per le strade extraurbane locali, ed i 50 km/h per le strade nei centri abitati, con la possibilità di elevare tale limite fino ad un massimo di 70 km/h per le strade urbane le cui caratteristiche costruttive e funzionali lo consentano, previa installazione degli appositi segnali. Sulle autostrade a tre corsie più corsia di emergenza per ogni senso di marcia dotate di apparecchiature debitamente omologate per il calcolo della velocità media di percorrenza su tratti determinati, gli enti proprietari o concessionari possono elevare il limite massimo di velocità fino a 150 km/h sulla base delle caratteristiche progettuali ed effettive del tracciato, previa installazione degli appositi segnali, sempreché lo consentano l’intensi